# <font color='violet'> Further Cleaning of Duplicate Reviews
Using prescription drug review initially wrangled here wrangled here: https://github.com/fractaldatalearning/psychedelic_efficacy/blob/main/notebooks/1-kl-wrangle-tabular.ipynb

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/interim/studies_initial_cleaning.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50637 entries, 0 to 50636
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  50637 non-null  int64  
 1   drug        50637 non-null  object 
 2   rating      50637 non-null  float64
 3   condition   50637 non-null  object 
 4   review      50637 non-null  object 
 5   date        50637 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.3+ MB


In [3]:
# Drop "Unnamed" column; it's redundant with the index
df = df.drop(columns=['Unnamed: 0'])
df.head(2)

,drug,rating,condition,review,date
0,vyvanse,9.0,add,I had began taking 20mg of Vyvanse for three m...,0
1,dextroamphetamine,8.0,add,Switched from Adderall to Dexedrine to compare...,0


During EDA, I discovered that many reviews are duplicated. It seems that what I discovered is one person may have just written one big review for all their drugs and entered it multiple times, with a different drug and rating each time. Is this behavior an outlier or are there many examples like this? 

<font color='violet'> Decide what to do about duplicated reviews. 

In [4]:
df[df.review.duplicated()==True]

,drug,rating,condition,review,date
668,Quetiapine,9.0,depression,"""been great for me except for the weight gain ...","October 23, 2016"
686,Buprenorphine / naloxone,1.0,addiction,"""I was on suboxone strips which was working gr...","June 28, 2017"
732,Desvenlafaxine,4.0,anxiety,"""I am into my 4th week of Pristiq and it hasn&...","October 8, 2011"
816,Suboxone,9.0,addiction,"""My personal experience with suboxone is good ...","May 27, 2017"
821,Lorazepam,8.0,anxiety,"""Most subtle of the benzos i have tried. Made...","October 28, 2013"
...,...,...,...,...,...
50631,Geodon,3.0,bipolar,"""I was in a very bad place at the time I start...","July 25, 2016"
50632,Venlafaxine,9.0,anxiety,"""Had panic attacks and social anxiety starting...","November 10, 2016"
50634,Ativan,9.0,anxiety,"""I was super against taking medication. I&#039...","August 16, 2016"
50635,Fluoxetine,8.0,ocd,"""I have been off Prozac for about 4 weeks now....","January 21, 2015"


Many rows actually contain duplicate reviews, each connected with multiple different drugs. Did the data start out this way, or did I make an error during initial wrangling?

In [ ]:
drugs_dotcom_train = pd.read_csv('../data/raw/drugsComTrain_raw.tsv', sep='\t')
drugs_dotcom_test = pd.read_csv('../data/raw/drugsComTest_raw.tsv', sep='\t')
druglib_train = pd.read_csv('../data/raw/drugLibTrain_raw.tsv', sep='\t')
druglib_test = pd.read_csv('../data/raw/drugLibTest_raw.tsv', sep='\t')
psytar = pd.read_csv('../data/raw/PsyTAR_dataset_samples.csv')

In [ ]:
# Make a function to help figure out what's going on 
def inspect_duplicate_reviews(df, column):
    df = df.sort_values(by=column)
    print(len(df), len(df[df[column].duplicated()==True]))
    return df[df[column].duplicated()==True].head()

# What my current working data looks like
inspect_duplicate_reviews(df, 'review')

In [ ]:
# Check out each of the other raw datasets
drugs_dotcom_train.info()

In [ ]:
inspect_duplicate_reviews(drugs_dotcom_train, 'review')

In [ ]:
# 30% of the original reviews from that set were duplicates. 
inspect_duplicate_reviews(drugs_dotcom_test, 'review')

In [ ]:
# 10% of drugs_dotcom_test was duplicates
druglib_train.info()

In [ ]:
inspect_duplicate_reviews(druglib_train, 'commentsReview')

In [ ]:
# Fewer of these were duplicates
psytar.info()

In [ ]:
inspect_duplicate_reviews(psytar, 'comment')

This last raw dataset has about 15% duplicate values but few rows overall. 

I did go back to the wrangling notebook and don't see any errors that would have caused this. I think I just didn't notice earlier because I would expect there to be duplicates in many of the columns (drug, condition) without it being a problem at all. Or perhaps completely duplicated rows, and took care of those. But it didn't cross my mind to think that specifically the reveiw column would have duplicates across multiple drugs. 

There are enough duplicated reviews in the raw data to account for all the duplicates in my current dataframe. My best working hypothesis is that the duplicate reviews appeared more often with psych meds because people may cycle through and try many drugs and then write up one big narrative to submit. Or perhaps, they feel one way about the drug's effects and go back to change their rating later, which results in two rows varying only by rating. I may need to more closely inspect each set of duplicates and find out which drugs the reviews are actually relevant for, removing the rest of the rows. 

<font color='violet'> Remove rows with irrelevant duplicated reviews

In [ ]:
# Start with just one set of duplicates and see what I find.
df.head(8)

It appears that somebody submitted the same review for vyvanse, dextroamphetamine, saizen, and zyprexa. And with vyvanse, they submitted it as being used to treat both add and adhd. And for add they gave it a rating of 9 with one submission and 10 with another. 

I can see already that this definitly pertains to vyvanse. Since the add ratings are ambiguous, I can just get rid of those and keep the row for adhd. 

In [ ]:
df = df.drop(labels=[0,5])
df.head(6)

In [ ]:
# Take a closer look at the full review to see if it pertains to the other drugs.
df.review[1]

In [ ]:
# This only pertains to vyvanse. Drop other rows. 
df = df.drop(labels=[1,3,4])
df.head(2)

How many sets of duplicates will I need to work with? 

In [ ]:
len(df[df.review.duplicated()==True]['review'].unique())

There are so many sets of duplicates, I'm going to need to find some way to do automated/batch deletion.

This could be a place to group by the review until there's just one row per review with various drug/rating/condition combinations that can be aggregated for each set of duplicates or analyzed more easily in batches for quicker identification of values to keep or delete. 

In [ ]:
# Create a columm where I can hold whether each row should be kept or deleted. 
# Work until every row is filled with a value, then delete indicated rows.
df['keep'] = ''
df.head()

<font color='violet'> First, mark for keeping any non-duplicate reviews

In [ ]:
df.loc[(df.review.duplicated(keep=False)==False),'keep'] = 'yes'
df[df.review.duplicated(keep=False)==False]

<font color='violet'> Mark for keeping any rows where the name of the drug is contained in the text of the review. 

In [ ]:
grouped_df = df.groupby(['review', 'drug']).count()
grouped_df

In [ ]:
# Row indices are defined by the drug column. Gather indices for reviews to keep.
grouped_df_indices_to_keep = []

# Find if the review column contains the string from the drug column.
for row in range(len(grouped_df.index)):
    if (grouped_df.index[row][1].lower() in grouped_df.index[row][0].lower()) == True:
        grouped_df_indices_to_keep.append(row)
        
grouped_df_indices_to_keep[:5]

In [ ]:
len(grouped_df_indices_to_keep)

In [ ]:
# It seems many rows should be kept. Check that this worked correctly.
grouped_df.index[1]

In [ ]:
# The drug name is in the review narrative. 
# Isolate just the rows to keep
grouped_to_keep = pd.MultiIndex.to_frame(grouped_df.index[grouped_df_indices_to_keep])
grouped_to_keep.head()

In [ ]:
grouped_to_keep = grouped_to_keep.reset_index(drop=True)
grouped_to_keep

In [ ]:
# This is the correct number of rows for reviews that contain the drug name
# Add the keep row so that this df can be merged with the original df
grouped_to_keep['keep'] = 'yes'
grouped_to_keep.head()

In [ ]:
df = df.merge(right=grouped_to_keep, how='left', on=['review', 'drug'])
df

In [ ]:
# This contains the correct number of rows to match the original df
# keep_y has the values I need for knowing which rows to keep so far

df = df.drop(columns=['keep_x'])
df.head()

In [ ]:
df = df.rename(columns={'keep_y':'keep'})
df.head()

In [ ]:
# Fill na in keep column to make it easier to work with later.
df['keep'] = df.keep.fillna('z')
df.head()

Dig further into rows where the name of the drug is not in the review. This does not necessarily mean the review isn't applicable to the associated drug. But, I'd say that if there is a review that contains a drug name, that same review should be dropped wherever it appears along with a different drug not mentioned. 

<font color='violet'> Drop rows where text doesn't contain drug name but drug name is present in the same review for a different drug. 

In [ ]:
no_drug_in_review = df.groupby(['review', 'keep']).count().sort_values(
    by=['review', 'keep'])
no_drug_in_review

In [ ]:
len(no_drug_in_review)

There are fewer indices this time because some rows have multiple drugs aggregated within the 'z' row for a review. If a review has only unknown (z) keep values, that should remain unknown for now. But if there is a yes row for the review, then that review's z's should be come no's. 

Specifically, identify reviews for rows to keep. Then, since yes comes before z in the sorting, the yes row is on top in each set of rows per review. So, the row directly below each yes row can be deleted, IF it has the same review. (If it doesn't have the same review, then it should remain unknown for now). 

In [ ]:
indices_to_drop = []

for idx in range(len(no_drug_in_review)):
    # Isolate reviews for rows to keep, and if  
    if (no_drug_in_review.index[idx][1] == 'yes' and no_drug_in_review.index[idx][0] == 
        no_drug_in_review.index[idx+1][0]):
        indices_to_drop.append(idx+1)

indices_to_drop[:5]

In [ ]:
len(indices_to_drop)

In [ ]:
# Confirm this worked correctly
no_drug_in_review.index[1]

In [ ]:
no_drug_in_review.index[2]

In [ ]:
# This worked correctly. Index 2 is slotted for dropping, and it has the same review as 
# index 1, which is labeled yes to keep. Now, isolate the rows to drop.

un_reviewed_to_drop = pd.MultiIndex.to_frame(no_drug_in_review.index[indices_to_drop])
un_reviewed_to_drop.head()

In [ ]:
un_reviewed_to_drop = un_reviewed_to_drop.reset_index(drop=True)
un_reviewed_to_drop.head()

In [ ]:
# Change keep value to no
un_reviewed_to_drop['keep'] = 'no'
un_reviewed_to_drop.head()

This can again be merged with df. There may be multiple drugs per "no keep" review, and that's okay; each one can be filled with no because these reviews should be dropped wherever they appear, since they already have an associated yes review that is definitely relevant to its associated drug. Wherever the new keep column says no but the old keep column says yes, the value should be yes.

In [ ]:
df = df.merge(right=un_reviewed_to_drop, on='review', how='left')
df

In [ ]:
# Now, if keep_x = yes, that's the row to keep for that review. 
# anyplace where keep_x = z but keep_y = no, the keep value should end up as no

for row in range(len(df)):
    if df.loc[row,'keep_y'] == 'no' and df.loc[row,'keep_x'] == 'z':
        df.loc[row,'keep_x'] = 'no'

df[df.keep_y=='no']

In [ ]:
# Check if this worked correctly
df[df.review == df.loc[122,'review']]

In [ ]:
# This looks correct. The drug name is in the review associated with the yes row
# The matching review now says no in keep_x. I can delete the row keep_y

df = df.drop(columns=['keep_y'])
df.head()

In [ ]:
df = df.rename(columns={'keep_x':'keep'})
df.head()

In [ ]:
# What remains? How many rows still have a keep value of z?
len(df[df.keep=='z'])

<font color='violet'> Deal with any reviews that are just duplicates related to multiple conditions.  

In [ ]:
grouped_by_condition = df.groupby(['review', 'condition']).count()
grouped_by_condition

In [ ]:
# Those duplicated by condition would show up where 2 subsequent indices have the same review.
indices_duplicated_by_condition = []
for idx in range(len(grouped_by_condition)):
    # Need to include a try-except since sometimes idx+1 won't exist
    try:
        if grouped_by_condition.index[idx][0] == grouped_by_condition.index[idx+1][0]:
            indices_duplicated_by_condition.append(idx)
            indices_duplicated_by_condition.append(idx+1)
    except: pass
        
indices_duplicated_by_condition[:5]    

In [ ]:
# Take a look at the rows I've identified
duplicated_by_condition = pd.MultiIndex.to_frame(grouped_by_condition.index[
    indices_duplicated_by_condition])
duplicated_by_condition

Here, I think it would make sense to just choose one of the conditions to keep. If there were many pairs like this, I might create columns "condition1" and "condition2", but if "condition2" would only have 4 values out of tens of thousands of rows, that seems like a waste. Instead, I'll go ahead and just keep the row for the less-common condition, so as to balance rather than further un-balance the condition column. 

First I'll need a dictionary of conditions

In [ ]:
conditions_rank = df.condition.value_counts().to_frame()
conditions_rank.head()

In [ ]:
conditions_rank['rank'] = range(len(conditions_rank))
conditions_rank.head()

In [ ]:
conditions_rank = conditions_rank.drop(columns=['condition']).reset_index().rename(
    columns={'index':'condition'})
conditions_rank.head()

In [ ]:
conditions_rank = conditions_rank.set_index('condition').to_dict()['rank']
conditions_rank

In [ ]:
# Prepare dataframe of just reviews that have multiple conditions attached
duplicated_by_condition = duplicated_by_condition.reset_index(drop=True)
duplicated_by_condition.head()

In [ ]:
# Get this in a format where the conditions for each review can be compared
for row in range(len(duplicated_by_condition)):
    duplicated_by_condition.loc[row,'rank'] = conditions_rank[duplicated_by_condition.loc[
        row, 'condition']]

duplicated_by_condition.head()

In [ ]:
# Identify max rank as the condition to keep for each review
condition_to_keep = duplicated_by_condition.groupby(['review']).max()
condition_to_keep.head()

In [ ]:
# This is the wrong condition listed, but the correct condition rank that should be kept.

condition_to_keep = condition_to_keep.drop(columns=['condition'])
condition_to_keep.head()

In [ ]:
# Change rank to int type
condition_to_keep['rank'] = condition_to_keep['rank'].astype(int)
condition_to_keep.head()

In [ ]:
# Create regular df to iterate through:
condition_to_keep = condition_to_keep.reset_index()
condition_to_keep.head()

In [ ]:
# Refill conditions 
for row in range(len(condition_to_keep)):
    for key, value in conditions_rank.items():
        if condition_to_keep.loc[row,'rank'] == value:
                condition_to_keep.loc[row,'condition'] = key
            
condition_to_keep.head()

In [ ]:
# These conditions should have a keep value of 'yes'
condition_to_keep['keep'] = 'yes'
condition_to_keep.head()

In [ ]:
# Merge with duplicated_by_condition so as to be able to mark remaining rows with "no"
duplicated_by_condition = duplicated_by_condition.merge(condition_to_keep, how='left')
duplicated_by_condition

In [ ]:
duplicated_by_condition = duplicated_by_condition.drop(columns=['rank']).fillna('no')
duplicated_by_condition.head()

In [ ]:
# Now duplicated_by_condition can be merged with the rest of the df
df = df.merge(duplicated_by_condition, on=['review', 'condition'], how='left')
df

In [ ]:
# How did that work? What does the first review with duplicated conditions look like?
df[df.review.str.contains('After many months spent being given ten')]

In [ ]:
# I'd previously mis-labeled some rows. 
df.sort_values(by=['keep_y', 'keep_x']).head(7)

In [ ]:
# Wherever keep_y is not null, that is the value that should be kept. 
# Otherwise keep the value of keep_y

df = df.reset_index(drop = True)
df.head()

In [ ]:
for row in range(len(df)):
    if df.loc[row,'keep_y'] == 'yes' or df.loc[row,'keep_y'] == 'no':
        df.loc[row,'keep'] = df.loc[row,'keep_y']
    else: df.loc[row,'keep'] = df.loc[row,'keep_x']
        
df.head()

In [ ]:
df.sort_values(by=['keep_y', 'keep_x']).head()

In [ ]:
# This looks correct so far. Clean up. 
df = df.drop(columns=['keep_x', 'keep_y'])
df.head()

Now, everywhere there is a duplicated review, a row for that review is being kept if it contains the drug name and it is submitted for the least-common condition. Reviews are marked for removal if they don't contain the name of the drug but their duplicate does. And being removed if submitted for a more-common condition where the review is also submitted for a less-common condition. 

But, wherever there is no drug name at all in the review, duplicates likely still exist across multiple drugs. This may be a place where new columns for drug1, drug2, drug3 may be necessary

<font color='violet'> Deal with remaining reviews duplicated across multiple drugs.

In [ ]:
# How many reviews remain to deal with?
len(df[(df.review.duplicated(keep=False)==True) & (df.keep=='z')])

In [ ]:
# What's the highest number of drugs associated with a single review?
row_count = df.groupby(['review']).count()
row_count.sort_values(by='drug', ascending=False)

The review "Good" is associated with 16 different drugs. Add columns drug0...drug15 wherever a review has more than one associated drug. First, sort drugs by prevalance, then enumerate drugs per review so that column can then become multiple nuew columns. Finally, create a pivot table and fill values of new drug_n columns with drug names.

In [ ]:
# Go back and sort drugs according to how common they are so they're enumerated that way
by_drug = df.groupby('drug').count().sort_values(by='rating', ascending=False)
by_drug

In [ ]:
by_drug['drug_prevalance'] = range(len(by_drug))
by_drug = by_drug.drop(columns=[
    'rating', 'condition', 'review', 'date', 'keep']).reset_index()
by_drug

In [ ]:
# Merge with df so that drugs have their prevalance values associated
df = df.merge(by_drug, how='left')
df

In [ ]:
# Create drug_n to enumerate drugs per review
df['drug_n'] = df.sort_values(by='drug_prevalance').groupby(['review']).cumcount()
df.sort_values(by=['review', 'drug_n'])
df.drug_n.max()

In [ ]:
# That appears to have worked. drug_n should contain values 0:15, for max 15 duplicates/review
# Now fill in values for some new drug_n columns
wide_df = pd.pivot(data=df, columns='drug_n', values='drug', index='review')
wide_df.head()

In [ ]:
wide_df.info()

In [ ]:
# drugs are now distributed across rows 0-15. Get this in a format to re-merge w/ full df
wide_df = wide_df.reset_index().rename(columns={0:'drug0', 1:'drug1', 2:'drug2', 3:'drug3', 
                                                4:'drug4', 5:'drug5', 6:'drug6', 7:'drug7', 
                                                8:'drug8', 9:'drug9', 10:'drug10', 11:'drug11', 
                                                12:'drug12', 13:'drug13', 14:'drug14', 
                                                15:'drug15'})
wide_df.head()

In [ ]:
drug_cols_df = df.merge(wide_df, on='review', how='left')
drug_cols_df

In [ ]:
# This has the correct number and type of rows and columns. Clean up columns. 
drug_cols_df = drug_cols_df.drop(columns=['drug', 'drug_prevalance', 'drug_n'])
drug_cols_df.info()

This is now in a format where there are (hopefully) completely duplicated rows. Reviews with duplicates and a keep value of z should now all have the same drugs associated with them, just spread over multiple columns. See if it works to simply drop completely duplicate rows. 

<font color='violet'> Delete duplicates and rows marked for deletion.

In [ ]:
drug_cols_df = drug_cols_df.drop_duplicates()
drug_cols_df.info()

In [ ]:
# That did get rid of 7k rows. 
drug_cols_df = drug_cols_df[drug_cols_df.keep!='no'].copy()
drug_cols_df.info()

In [ ]:
# Another 10k rows taken care of. Check out what's up now with duplicated reviews
len(drug_cols_df[drug_cols_df.review.duplicated(keep=False)==True])

In [ ]:
# This is very easy to deal with now
drug_cols_df[drug_cols_df.review.duplicated(keep=False)==True].sort_values(by='review')

Remaining duplicates were reviews that were either identical and submitted on two different dates or varied only by their rating. I'll just keep the latest review. 

<font color='violet'> Nuke remaining duplicate reviews

In [ ]:
rows_to_drop = [28271, 17638, 3750, 919, 42884, 5937, 972, 31390]
final_df = drug_cols_df.drop(index=rows_to_drop).drop(columns=['keep'])
final_df.info()

There are null values here, but they are truly null. They'll need to be changed prior to modeling, but for the purposes of EDA they should be kept. This should finally be ready to use for EDA. Pick that up in the next notebook: 

In [ ]:
final_df.to_csv('../data/interim/studies_no_duplicates.csv')